In [ ]:
import utils, numpy, pandas, tensorflow, os, sys
from tensorflow import keras
# tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
sys.modules.pop('utils')
import utils

# Load Model

In [ ]:
max_tweet_length = 106    # from training
inputA = keras.layers.Input(shape = (max_tweet_length,1), name = 'tokenised_tweets')
x = keras.layers.LSTM(128, return_sequences=True)(inputA)
x = keras.layers.LSTM(64)(x)
# x = LSTM(128)(inputA)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation='relu')(x)


inputB = keras.layers.Input(shape=(4,1), name='others')
y = keras.layers.Flatten()(inputB)
y = keras.layers.Dense(4, activation='relu')(y)

combined = keras.layers.concatenate([x,y])
output = keras.layers.Dense(32, activation='relu')(combined)
output = keras.layers.Dropout(0.2)(output)
output = keras.layers.Dense(1, activation='sigmoid')(output)
lstm_model_2input = keras.models.Model([inputA, inputB], output)

lstm_model_2input.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


lstm_model_2input.load_weights("LSTM_model_02_969.h5")

# Prepare data

In [ ]:
# Quality examples
# single_input = {'tweet': ['aaaaa trump win twitter cash'],'following': [100], 'followers': [200], 'actions': [33], 'is_retweet':[1] }
single_input = {'tweet': ["It's the everything else that's complicated. #PESummit #PXpic.twitter.com/Jsv6BAFQMl"],'following': [0], 'followers': [11500], 'actions': [0], 'is_retweet':[0] }


# Spam examples
# single_input = {'tweet': ['EBMUD ending penalties for excessive water users https://t.co/D5a1FMVMHd'],'following': [4435], 'followers': [16000], 'actions': [27000], 'is_retweet':[0] }
# single_input = {'tweet': ['aaaaaaaaaaaaaaaaaaaaa hell trump'],'following': [8221], 'followers': [4553], 'actions': [27858], 'is_retweet':[0] }


columns_to_be_standardised = ['following', 'followers', 'actions']
input_df = pandas.DataFrame(data=single_input)
sigma, mu, tokenizer = utils.load_pickle('sigma_train.pkl'), utils.load_pickle('mu_train.pkl'), utils.load_pickle('tokeniser_train.pkl')

# Process 'tweet'
tweet_las, tweet_tokenised = utils.tokenise(input_df['tweet'], tokenizer)
tweet_tensor = tensorflow.convert_to_tensor(tweet_tokenised)
print(tweet_las)
print(tweet_tokenised)
print(tweet_tokenised.shape, tweet_tensor.shape)

# Process 'following' 'followers' 'actions' 'is_retweet'
others_col_std = utils.standardise(input_df, mu, sigma, columns_to_be_standardised)
others_col_std.drop(['tweet'], axis=1, inplace=True)
print(others_col_std)

# Predict

## Spam is 1, Not Spam is 0

In [ ]:
(lstm_model_2input.predict([tweet_tensor, others_col_std]) >= 0.5) * 1  